In [217]:
# compare ensemble to each baseline classifier
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
import pandas as pd

In [218]:
df = pd.read_csv('train.csv')
df['label'] = df['label'].map({'correct': 1, 'incorrect': 0})
df = df.dropna()
df = df.drop(['pickup_time','drop_time'], axis = 1)

In [219]:
import math
import numpy as np
def deg_to_rad(dr):
    return (dr*math.pi)/180

df['pick_lat'] = deg_to_rad(df['pick_lat'])
df['pick_lon'] = deg_to_rad(df['pick_lon'])
df['drop_lat'] = deg_to_rad(df['drop_lat'])
df['drop_lon'] = deg_to_rad(df['drop_lon'])
dlon = df['drop_lon'] - df['pick_lon']
dlat = df['drop_lat'] - df['pick_lat']
df['dist'] = np.sin(dlat/2)**2 + np.cos(df['pick_lat'])*np.cos(df['drop_lat'])*np.sin(dlon/2)**2
df['dist'] = 2*np.arctan2(df['dist']**0.5,(1-df['dist'])**0.5)
R = 6373.0
df['dist'] = R*df['dist']

In [220]:
df = df.drop(['tripid','pick_lat','pick_lon','drop_lat','drop_lon'], axis = 1) 

In [221]:
y = df['label']
X = df.drop(['label'],axis=1)

In [222]:
X

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,dist
0,10.5,834.0,56.0,0.0000,64.0,270.32,5.094369
1,10.5,791.0,47.0,0.0000,134.0,197.85,3.169052
2,10.5,1087.0,80.0,0.0000,61.0,301.64,6.307375
3,10.5,598.0,271.0,15.6638,68.0,82.30,0.862217
5,10.5,3407.0,182.0,0.0000,112.0,1065.02,24.214638
...,...,...,...,...,...,...,...
17171,10.5,838.0,93.0,5.4219,451.0,198.26,2.106037
17172,10.5,2151.0,428.0,0.0000,39.0,581.23,10.871789
17173,10.5,263.0,9.0,0.0000,110.0,76.20,1.045847
17174,10.5,858.0,115.0,0.0000,317.0,133.31,2.879981


In [290]:
from xgboost import XGBClassifier
# get a stacking ensemble of models
from sklearn.ensemble import VotingClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.datasets import load_iris 
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split 
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
  
# loading iris dataset 

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3) 
  

In [291]:
ran = RandomForestClassifier(n_estimators=600,criterion='gini', max_depth=None, 
                             min_samples_split=2, min_samples_leaf=2, min_weight_fraction_leaf=0.0, 
                             max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, 
                             min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=6, 
                             random_state=None, verbose=0, warm_start=False, class_weight=None, 
                             ccp_alpha=0.0, max_samples=None)
ran.fit(X_train, y_train) 
y_pred = ran.predict(X_test)
ran_score = f1_score(y_test, y_pred)
print("Random Forrest F1 score = {}".format(f1_score(y_test, y_pred)))

Random Forrest F1 score = 0.9766013763896242


In [301]:
cat = CatBoostClassifier()
cat.fit(X_train, y_train) 
y_pred = cat.predict(X_test)

Learning rate set to 0.029637
0:	learn: 0.6570366	total: 9.7ms	remaining: 9.69s
1:	learn: 0.6288450	total: 14.2ms	remaining: 7.09s
2:	learn: 0.6026680	total: 17.7ms	remaining: 5.88s
3:	learn: 0.5753113	total: 21.2ms	remaining: 5.29s
4:	learn: 0.5518779	total: 24.7ms	remaining: 4.91s
5:	learn: 0.5288415	total: 28.2ms	remaining: 4.67s
6:	learn: 0.5062288	total: 31.8ms	remaining: 4.51s
7:	learn: 0.4873513	total: 35.6ms	remaining: 4.41s
8:	learn: 0.4674763	total: 39.2ms	remaining: 4.31s
9:	learn: 0.4529508	total: 42.8ms	remaining: 4.24s
10:	learn: 0.4350544	total: 46.4ms	remaining: 4.17s
11:	learn: 0.4193743	total: 50.2ms	remaining: 4.13s
12:	learn: 0.4043497	total: 53.8ms	remaining: 4.08s
13:	learn: 0.3929824	total: 57.3ms	remaining: 4.03s
14:	learn: 0.3820341	total: 60.7ms	remaining: 3.99s
15:	learn: 0.3695318	total: 64.2ms	remaining: 3.95s
16:	learn: 0.3574675	total: 67.7ms	remaining: 3.92s
17:	learn: 0.3480839	total: 71.3ms	remaining: 3.89s
18:	learn: 0.3373460	total: 74.8ms	remaining:

194:	learn: 0.1538707	total: 790ms	remaining: 3.26s
195:	learn: 0.1535330	total: 803ms	remaining: 3.29s
196:	learn: 0.1534073	total: 811ms	remaining: 3.31s
197:	learn: 0.1532628	total: 815ms	remaining: 3.3s
198:	learn: 0.1531533	total: 822ms	remaining: 3.31s
199:	learn: 0.1529514	total: 826ms	remaining: 3.3s
200:	learn: 0.1526401	total: 830ms	remaining: 3.3s
201:	learn: 0.1524682	total: 840ms	remaining: 3.32s
202:	learn: 0.1520949	total: 845ms	remaining: 3.32s
203:	learn: 0.1518363	total: 849ms	remaining: 3.31s
204:	learn: 0.1516339	total: 853ms	remaining: 3.31s
205:	learn: 0.1513756	total: 859ms	remaining: 3.31s
206:	learn: 0.1512842	total: 863ms	remaining: 3.31s
207:	learn: 0.1511329	total: 868ms	remaining: 3.31s
208:	learn: 0.1509749	total: 872ms	remaining: 3.3s
209:	learn: 0.1508733	total: 876ms	remaining: 3.29s
210:	learn: 0.1506665	total: 880ms	remaining: 3.29s
211:	learn: 0.1503393	total: 886ms	remaining: 3.29s
212:	learn: 0.1502127	total: 891ms	remaining: 3.29s
213:	learn: 0.15

355:	learn: 0.1327723	total: 1.81s	remaining: 3.28s
356:	learn: 0.1326860	total: 1.82s	remaining: 3.27s
357:	learn: 0.1325118	total: 1.82s	remaining: 3.27s
358:	learn: 0.1323617	total: 1.83s	remaining: 3.26s
359:	learn: 0.1322794	total: 1.83s	remaining: 3.26s
360:	learn: 0.1322193	total: 1.84s	remaining: 3.25s
361:	learn: 0.1321041	total: 1.84s	remaining: 3.25s
362:	learn: 0.1319517	total: 1.85s	remaining: 3.25s
363:	learn: 0.1318901	total: 1.85s	remaining: 3.24s
364:	learn: 0.1317501	total: 1.86s	remaining: 3.23s
365:	learn: 0.1316455	total: 1.86s	remaining: 3.23s
366:	learn: 0.1315851	total: 1.87s	remaining: 3.22s
367:	learn: 0.1314224	total: 1.87s	remaining: 3.22s
368:	learn: 0.1313308	total: 1.88s	remaining: 3.21s
369:	learn: 0.1311448	total: 1.88s	remaining: 3.2s
370:	learn: 0.1310475	total: 1.89s	remaining: 3.2s
371:	learn: 0.1309971	total: 1.89s	remaining: 3.19s
372:	learn: 0.1309304	total: 1.89s	remaining: 3.18s
373:	learn: 0.1308852	total: 1.9s	remaining: 3.18s
374:	learn: 0.1

547:	learn: 0.1160785	total: 2.61s	remaining: 2.15s
548:	learn: 0.1160018	total: 2.62s	remaining: 2.15s
549:	learn: 0.1158710	total: 2.62s	remaining: 2.14s
550:	learn: 0.1158254	total: 2.62s	remaining: 2.14s
551:	learn: 0.1157550	total: 2.63s	remaining: 2.13s
552:	learn: 0.1156585	total: 2.63s	remaining: 2.13s
553:	learn: 0.1155673	total: 2.64s	remaining: 2.12s
554:	learn: 0.1155204	total: 2.64s	remaining: 2.12s
555:	learn: 0.1154294	total: 2.64s	remaining: 2.11s
556:	learn: 0.1153960	total: 2.65s	remaining: 2.1s
557:	learn: 0.1153230	total: 2.65s	remaining: 2.1s
558:	learn: 0.1152512	total: 2.65s	remaining: 2.09s
559:	learn: 0.1151797	total: 2.66s	remaining: 2.09s
560:	learn: 0.1150444	total: 2.66s	remaining: 2.08s
561:	learn: 0.1149254	total: 2.67s	remaining: 2.08s
562:	learn: 0.1148892	total: 2.67s	remaining: 2.07s
563:	learn: 0.1148214	total: 2.67s	remaining: 2.07s
564:	learn: 0.1147429	total: 2.68s	remaining: 2.06s
565:	learn: 0.1147094	total: 2.68s	remaining: 2.05s
566:	learn: 0.

733:	learn: 0.1047630	total: 3.39s	remaining: 1.23s
734:	learn: 0.1047344	total: 3.4s	remaining: 1.23s
735:	learn: 0.1046787	total: 3.41s	remaining: 1.22s
736:	learn: 0.1046084	total: 3.41s	remaining: 1.22s
737:	learn: 0.1045382	total: 3.42s	remaining: 1.21s
738:	learn: 0.1044886	total: 3.42s	remaining: 1.21s
739:	learn: 0.1044397	total: 3.42s	remaining: 1.2s
740:	learn: 0.1044038	total: 3.43s	remaining: 1.2s
741:	learn: 0.1043371	total: 3.43s	remaining: 1.19s
742:	learn: 0.1042794	total: 3.43s	remaining: 1.19s
743:	learn: 0.1042242	total: 3.44s	remaining: 1.18s
744:	learn: 0.1041806	total: 3.44s	remaining: 1.18s
745:	learn: 0.1041494	total: 3.44s	remaining: 1.17s
746:	learn: 0.1041022	total: 3.45s	remaining: 1.17s
747:	learn: 0.1040284	total: 3.45s	remaining: 1.16s
748:	learn: 0.1039709	total: 3.46s	remaining: 1.16s
749:	learn: 0.1039523	total: 3.46s	remaining: 1.15s
750:	learn: 0.1038672	total: 3.46s	remaining: 1.15s
751:	learn: 0.1038420	total: 3.47s	remaining: 1.14s
752:	learn: 0.1

933:	learn: 0.0948201	total: 4.17s	remaining: 295ms
934:	learn: 0.0948061	total: 4.18s	remaining: 291ms
935:	learn: 0.0947381	total: 4.2s	remaining: 287ms
936:	learn: 0.0947091	total: 4.2s	remaining: 283ms
937:	learn: 0.0946777	total: 4.21s	remaining: 278ms
938:	learn: 0.0946448	total: 4.21s	remaining: 273ms
939:	learn: 0.0946314	total: 4.21s	remaining: 269ms
940:	learn: 0.0945598	total: 4.22s	remaining: 264ms
941:	learn: 0.0944592	total: 4.22s	remaining: 260ms
942:	learn: 0.0944088	total: 4.22s	remaining: 255ms
943:	learn: 0.0943975	total: 4.23s	remaining: 251ms
944:	learn: 0.0943598	total: 4.23s	remaining: 246ms
945:	learn: 0.0942957	total: 4.24s	remaining: 242ms
946:	learn: 0.0942527	total: 4.24s	remaining: 237ms
947:	learn: 0.0942326	total: 4.25s	remaining: 233ms
948:	learn: 0.0941973	total: 4.25s	remaining: 228ms
949:	learn: 0.0941427	total: 4.25s	remaining: 224ms
950:	learn: 0.0941170	total: 4.26s	remaining: 219ms
951:	learn: 0.0940635	total: 4.26s	remaining: 215ms
952:	learn: 0.

In [302]:
cat_score = f1_score(y_test, y_pred)
print("Catboost F1 score = {}".format(f1_score(y_test, y_pred)))

Catboost F1 score = 0.977895855472901


In [314]:
xg = XGBClassifier(n_estimators=1200,gamma=0.6,learning_rate=0.01)
xg.fit(X_train, y_train) 
y_pred = xg.predict(X_test)
xg_score = f1_score(y_test, y_pred)
print("XGboost F1 score = {}".format(f1_score(y_test, y_pred)))

XGboost F1 score = 0.9770895205770047


In [316]:
# group / ensemble of models 
estimator = [] 
estimator.append(('ran', ran)) 
estimator.append(('cat', cat)) 
estimator.append(('xg', xg))

In [317]:
vot_hard = VotingClassifier(estimators = estimator, voting ='hard') 
vot_hard.fit(X_train, y_train) 
y_pred = vot_hard.predict(X_test) 

Learning rate set to 0.029637
0:	learn: 0.6570366	total: 8.74ms	remaining: 8.73s
1:	learn: 0.6288450	total: 12.8ms	remaining: 6.38s
2:	learn: 0.6026680	total: 16.3ms	remaining: 5.42s
3:	learn: 0.5753113	total: 19.8ms	remaining: 4.94s
4:	learn: 0.5518779	total: 23.3ms	remaining: 4.64s
5:	learn: 0.5288415	total: 26.9ms	remaining: 4.46s
6:	learn: 0.5062288	total: 30.8ms	remaining: 4.37s
7:	learn: 0.4873513	total: 34.9ms	remaining: 4.33s
8:	learn: 0.4674763	total: 38.4ms	remaining: 4.23s
9:	learn: 0.4529508	total: 41.8ms	remaining: 4.14s
10:	learn: 0.4350544	total: 45.2ms	remaining: 4.07s
11:	learn: 0.4193743	total: 48.7ms	remaining: 4.01s
12:	learn: 0.4043497	total: 52.2ms	remaining: 3.96s
13:	learn: 0.3929824	total: 55.7ms	remaining: 3.92s
14:	learn: 0.3820341	total: 59.1ms	remaining: 3.88s
15:	learn: 0.3695318	total: 62.5ms	remaining: 3.85s
16:	learn: 0.3574675	total: 66.1ms	remaining: 3.82s
17:	learn: 0.3480839	total: 69.6ms	remaining: 3.79s
18:	learn: 0.3373460	total: 73ms	remaining: 

173:	learn: 0.1578451	total: 793ms	remaining: 3.77s
174:	learn: 0.1575747	total: 805ms	remaining: 3.79s
175:	learn: 0.1573710	total: 811ms	remaining: 3.79s
176:	learn: 0.1571563	total: 815ms	remaining: 3.79s
177:	learn: 0.1569759	total: 819ms	remaining: 3.78s
178:	learn: 0.1568934	total: 823ms	remaining: 3.78s
179:	learn: 0.1566402	total: 827ms	remaining: 3.77s
180:	learn: 0.1563458	total: 831ms	remaining: 3.76s
181:	learn: 0.1561130	total: 835ms	remaining: 3.75s
182:	learn: 0.1560060	total: 839ms	remaining: 3.75s
183:	learn: 0.1556630	total: 843ms	remaining: 3.74s
184:	learn: 0.1555212	total: 847ms	remaining: 3.73s
185:	learn: 0.1552761	total: 851ms	remaining: 3.72s
186:	learn: 0.1550345	total: 855ms	remaining: 3.71s
187:	learn: 0.1549562	total: 858ms	remaining: 3.71s
188:	learn: 0.1548585	total: 862ms	remaining: 3.7s
189:	learn: 0.1547134	total: 866ms	remaining: 3.69s
190:	learn: 0.1546435	total: 870ms	remaining: 3.68s
191:	learn: 0.1543685	total: 874ms	remaining: 3.68s
192:	learn: 0

367:	learn: 0.1314224	total: 1.58s	remaining: 2.71s
368:	learn: 0.1313308	total: 1.59s	remaining: 2.72s
369:	learn: 0.1311448	total: 1.6s	remaining: 2.72s
370:	learn: 0.1310475	total: 1.6s	remaining: 2.71s
371:	learn: 0.1309971	total: 1.61s	remaining: 2.71s
372:	learn: 0.1309304	total: 1.61s	remaining: 2.71s
373:	learn: 0.1308852	total: 1.61s	remaining: 2.7s
374:	learn: 0.1308232	total: 1.62s	remaining: 2.7s
375:	learn: 0.1307286	total: 1.62s	remaining: 2.69s
376:	learn: 0.1306041	total: 1.63s	remaining: 2.69s
377:	learn: 0.1304490	total: 1.63s	remaining: 2.69s
378:	learn: 0.1303904	total: 1.64s	remaining: 2.68s
379:	learn: 0.1303285	total: 1.64s	remaining: 2.68s
380:	learn: 0.1302558	total: 1.65s	remaining: 2.67s
381:	learn: 0.1301886	total: 1.65s	remaining: 2.67s
382:	learn: 0.1300767	total: 1.66s	remaining: 2.67s
383:	learn: 0.1300087	total: 1.66s	remaining: 2.66s
384:	learn: 0.1299172	total: 1.66s	remaining: 2.66s
385:	learn: 0.1298341	total: 1.67s	remaining: 2.65s
386:	learn: 0.12

546:	learn: 0.1161482	total: 2.58s	remaining: 2.13s
547:	learn: 0.1160785	total: 2.59s	remaining: 2.14s
548:	learn: 0.1160018	total: 2.6s	remaining: 2.13s
549:	learn: 0.1158710	total: 2.6s	remaining: 2.13s
550:	learn: 0.1158254	total: 2.61s	remaining: 2.13s
551:	learn: 0.1157550	total: 2.61s	remaining: 2.12s
552:	learn: 0.1156585	total: 2.62s	remaining: 2.11s
553:	learn: 0.1155673	total: 2.62s	remaining: 2.11s
554:	learn: 0.1155204	total: 2.62s	remaining: 2.1s
555:	learn: 0.1154294	total: 2.63s	remaining: 2.1s
556:	learn: 0.1153960	total: 2.63s	remaining: 2.09s
557:	learn: 0.1153230	total: 2.63s	remaining: 2.09s
558:	learn: 0.1152512	total: 2.64s	remaining: 2.08s
559:	learn: 0.1151797	total: 2.64s	remaining: 2.08s
560:	learn: 0.1150444	total: 2.65s	remaining: 2.07s
561:	learn: 0.1149254	total: 2.65s	remaining: 2.06s
562:	learn: 0.1148892	total: 2.65s	remaining: 2.06s
563:	learn: 0.1148214	total: 2.66s	remaining: 2.06s
564:	learn: 0.1147429	total: 2.66s	remaining: 2.05s
565:	learn: 0.11

736:	learn: 0.1046084	total: 3.37s	remaining: 1.2s
737:	learn: 0.1045382	total: 3.38s	remaining: 1.2s
738:	learn: 0.1044886	total: 3.39s	remaining: 1.2s
739:	learn: 0.1044397	total: 3.4s	remaining: 1.19s
740:	learn: 0.1044038	total: 3.4s	remaining: 1.19s
741:	learn: 0.1043371	total: 3.4s	remaining: 1.18s
742:	learn: 0.1042794	total: 3.41s	remaining: 1.18s
743:	learn: 0.1042242	total: 3.41s	remaining: 1.17s
744:	learn: 0.1041806	total: 3.42s	remaining: 1.17s
745:	learn: 0.1041494	total: 3.42s	remaining: 1.16s
746:	learn: 0.1041022	total: 3.42s	remaining: 1.16s
747:	learn: 0.1040284	total: 3.43s	remaining: 1.16s
748:	learn: 0.1039709	total: 3.43s	remaining: 1.15s
749:	learn: 0.1039523	total: 3.44s	remaining: 1.15s
750:	learn: 0.1038672	total: 3.44s	remaining: 1.14s
751:	learn: 0.1038420	total: 3.44s	remaining: 1.14s
752:	learn: 0.1038128	total: 3.45s	remaining: 1.13s
753:	learn: 0.1037834	total: 3.45s	remaining: 1.13s
754:	learn: 0.1037035	total: 3.46s	remaining: 1.12s
755:	learn: 0.1036

918:	learn: 0.0954609	total: 4.15s	remaining: 366ms
919:	learn: 0.0953923	total: 4.17s	remaining: 362ms
920:	learn: 0.0953690	total: 4.17s	remaining: 358ms
921:	learn: 0.0953547	total: 4.18s	remaining: 353ms
922:	learn: 0.0953393	total: 4.18s	remaining: 349ms
923:	learn: 0.0953161	total: 4.18s	remaining: 344ms
924:	learn: 0.0952194	total: 4.19s	remaining: 340ms
925:	learn: 0.0952047	total: 4.2s	remaining: 335ms
926:	learn: 0.0951468	total: 4.2s	remaining: 331ms
927:	learn: 0.0950943	total: 4.2s	remaining: 326ms
928:	learn: 0.0950560	total: 4.21s	remaining: 322ms
929:	learn: 0.0950016	total: 4.21s	remaining: 317ms
930:	learn: 0.0949793	total: 4.22s	remaining: 313ms
931:	learn: 0.0949319	total: 4.22s	remaining: 308ms
932:	learn: 0.0948783	total: 4.22s	remaining: 303ms
933:	learn: 0.0948201	total: 4.23s	remaining: 299ms
934:	learn: 0.0948061	total: 4.23s	remaining: 294ms
935:	learn: 0.0947381	total: 4.24s	remaining: 290ms
936:	learn: 0.0947091	total: 4.24s	remaining: 285ms
937:	learn: 0.0

In [318]:
# using accuracy_score metric to predict accuracy
hard_score = f1_score(y_test, y_pred)
print("Hard voting F1 score = {}".format(f1_score(y_test, y_pred)))

Hard voting F1 score = 0.9779333757691492


In [319]:
# Voting Classifier with soft voting 
vot_soft = VotingClassifier(estimators = estimator, voting ='soft') 
vot_soft.fit(X_train, y_train) 
y_pred = vot_soft.predict(X_test)  
# using accuracy_score 

Learning rate set to 0.029637
0:	learn: 0.6570366	total: 9.94ms	remaining: 9.93s
1:	learn: 0.6288450	total: 13.9ms	remaining: 6.96s
2:	learn: 0.6026680	total: 17.4ms	remaining: 5.78s
3:	learn: 0.5753113	total: 20.9ms	remaining: 5.2s
4:	learn: 0.5518779	total: 24.3ms	remaining: 4.84s
5:	learn: 0.5288415	total: 27.8ms	remaining: 4.6s
6:	learn: 0.5062288	total: 31.4ms	remaining: 4.46s
7:	learn: 0.4873513	total: 34.9ms	remaining: 4.33s
8:	learn: 0.4674763	total: 38.4ms	remaining: 4.22s
9:	learn: 0.4529508	total: 41.8ms	remaining: 4.13s
10:	learn: 0.4350544	total: 45.2ms	remaining: 4.06s
11:	learn: 0.4193743	total: 48.8ms	remaining: 4.02s
12:	learn: 0.4043497	total: 52.4ms	remaining: 3.98s
13:	learn: 0.3929824	total: 56ms	remaining: 3.95s
14:	learn: 0.3820341	total: 59.6ms	remaining: 3.91s
15:	learn: 0.3695318	total: 63.2ms	remaining: 3.89s
16:	learn: 0.3574675	total: 67.2ms	remaining: 3.88s
17:	learn: 0.3480839	total: 71.2ms	remaining: 3.88s
18:	learn: 0.3373460	total: 74.8ms	remaining: 3.

203:	learn: 0.1518363	total: 982ms	remaining: 3.83s
204:	learn: 0.1516339	total: 991ms	remaining: 3.84s
205:	learn: 0.1513756	total: 995ms	remaining: 3.84s
206:	learn: 0.1512842	total: 999ms	remaining: 3.83s
207:	learn: 0.1511329	total: 1s	remaining: 3.82s
208:	learn: 0.1509749	total: 1.01s	remaining: 3.81s
209:	learn: 0.1508733	total: 1.02s	remaining: 3.84s
210:	learn: 0.1506665	total: 1.03s	remaining: 3.86s
211:	learn: 0.1503393	total: 1.04s	remaining: 3.88s
212:	learn: 0.1502127	total: 1.05s	remaining: 3.88s
213:	learn: 0.1500904	total: 1.05s	remaining: 3.87s
214:	learn: 0.1499465	total: 1.06s	remaining: 3.87s
215:	learn: 0.1497923	total: 1.06s	remaining: 3.86s
216:	learn: 0.1496208	total: 1.07s	remaining: 3.86s
217:	learn: 0.1494670	total: 1.07s	remaining: 3.85s
218:	learn: 0.1493532	total: 1.08s	remaining: 3.84s
219:	learn: 0.1492198	total: 1.08s	remaining: 3.83s
220:	learn: 0.1490629	total: 1.08s	remaining: 3.82s
221:	learn: 0.1488812	total: 1.09s	remaining: 3.81s
222:	learn: 0.1

370:	learn: 0.1310475	total: 1.76s	remaining: 2.99s
371:	learn: 0.1309971	total: 1.77s	remaining: 2.98s
372:	learn: 0.1309304	total: 1.78s	remaining: 2.99s
373:	learn: 0.1308852	total: 1.78s	remaining: 2.99s
374:	learn: 0.1308232	total: 1.79s	remaining: 2.98s
375:	learn: 0.1307286	total: 1.79s	remaining: 2.98s
376:	learn: 0.1306041	total: 1.8s	remaining: 2.97s
377:	learn: 0.1304490	total: 1.8s	remaining: 2.97s
378:	learn: 0.1303904	total: 1.81s	remaining: 2.96s
379:	learn: 0.1303285	total: 1.81s	remaining: 2.96s
380:	learn: 0.1302558	total: 1.82s	remaining: 2.95s
381:	learn: 0.1301886	total: 1.82s	remaining: 2.94s
382:	learn: 0.1300767	total: 1.82s	remaining: 2.94s
383:	learn: 0.1300087	total: 1.83s	remaining: 2.94s
384:	learn: 0.1299172	total: 1.83s	remaining: 2.93s
385:	learn: 0.1298341	total: 1.84s	remaining: 2.92s
386:	learn: 0.1297559	total: 1.84s	remaining: 2.92s
387:	learn: 0.1296608	total: 1.84s	remaining: 2.91s
388:	learn: 0.1294641	total: 1.85s	remaining: 2.91s
389:	learn: 0.

544:	learn: 0.1162475	total: 2.56s	remaining: 2.14s
545:	learn: 0.1162226	total: 2.56s	remaining: 2.13s
546:	learn: 0.1161482	total: 2.57s	remaining: 2.13s
547:	learn: 0.1160785	total: 2.57s	remaining: 2.12s
548:	learn: 0.1160018	total: 2.58s	remaining: 2.12s
549:	learn: 0.1158710	total: 2.58s	remaining: 2.11s
550:	learn: 0.1158254	total: 2.59s	remaining: 2.11s
551:	learn: 0.1157550	total: 2.59s	remaining: 2.1s
552:	learn: 0.1156585	total: 2.6s	remaining: 2.1s
553:	learn: 0.1155673	total: 2.62s	remaining: 2.11s
554:	learn: 0.1155204	total: 2.62s	remaining: 2.1s
555:	learn: 0.1154294	total: 2.63s	remaining: 2.1s
556:	learn: 0.1153960	total: 2.63s	remaining: 2.09s
557:	learn: 0.1153230	total: 2.63s	remaining: 2.09s
558:	learn: 0.1152512	total: 2.64s	remaining: 2.08s
559:	learn: 0.1151797	total: 2.64s	remaining: 2.08s
560:	learn: 0.1150444	total: 2.65s	remaining: 2.07s
561:	learn: 0.1149254	total: 2.65s	remaining: 2.07s
562:	learn: 0.1148892	total: 2.65s	remaining: 2.06s
563:	learn: 0.114

714:	learn: 0.1058054	total: 3.35s	remaining: 1.33s
715:	learn: 0.1057343	total: 3.36s	remaining: 1.33s
716:	learn: 0.1056667	total: 3.37s	remaining: 1.33s
717:	learn: 0.1056005	total: 3.37s	remaining: 1.32s
718:	learn: 0.1055540	total: 3.38s	remaining: 1.32s
719:	learn: 0.1054460	total: 3.38s	remaining: 1.32s
720:	learn: 0.1053826	total: 3.39s	remaining: 1.31s
721:	learn: 0.1053066	total: 3.39s	remaining: 1.31s
722:	learn: 0.1052487	total: 3.4s	remaining: 1.3s
723:	learn: 0.1051556	total: 3.4s	remaining: 1.3s
724:	learn: 0.1051093	total: 3.41s	remaining: 1.29s
725:	learn: 0.1050791	total: 3.41s	remaining: 1.29s
726:	learn: 0.1050090	total: 3.42s	remaining: 1.28s
727:	learn: 0.1049874	total: 3.42s	remaining: 1.28s
728:	learn: 0.1049507	total: 3.44s	remaining: 1.28s
729:	learn: 0.1049157	total: 3.44s	remaining: 1.27s
730:	learn: 0.1048560	total: 3.44s	remaining: 1.27s
731:	learn: 0.1048294	total: 3.45s	remaining: 1.26s
732:	learn: 0.1047866	total: 3.46s	remaining: 1.26s
733:	learn: 0.10

882:	learn: 0.0972733	total: 4.14s	remaining: 549ms
883:	learn: 0.0971972	total: 4.15s	remaining: 545ms
884:	learn: 0.0971250	total: 4.16s	remaining: 540ms
885:	learn: 0.0970345	total: 4.16s	remaining: 535ms
886:	learn: 0.0969925	total: 4.16s	remaining: 530ms
887:	learn: 0.0969368	total: 4.17s	remaining: 526ms
888:	learn: 0.0969016	total: 4.17s	remaining: 521ms
889:	learn: 0.0968545	total: 4.17s	remaining: 516ms
890:	learn: 0.0967939	total: 4.18s	remaining: 511ms
891:	learn: 0.0967490	total: 4.18s	remaining: 506ms
892:	learn: 0.0966909	total: 4.19s	remaining: 502ms
893:	learn: 0.0965779	total: 4.19s	remaining: 497ms
894:	learn: 0.0965556	total: 4.19s	remaining: 492ms
895:	learn: 0.0965393	total: 4.2s	remaining: 487ms
896:	learn: 0.0965142	total: 4.2s	remaining: 482ms
897:	learn: 0.0964831	total: 4.21s	remaining: 478ms
898:	learn: 0.0964404	total: 4.21s	remaining: 473ms
899:	learn: 0.0963731	total: 4.21s	remaining: 468ms
900:	learn: 0.0963519	total: 4.22s	remaining: 463ms
901:	learn: 0.

In [320]:
score = accuracy_score(y_test, y_pred)
soft_score = f1_score(y_test, y_pred)
print("Soft voting score = {}".format(f1_score(y_test, y_pred)))

Soft voting score = 0.9776222292926079


In [321]:
print("Ran score = {}".format(ran_score))
print("Cat score = {}".format(cat_score))
print("XG score = {}".format(xg_score))
print("Hard voting score = {}".format(hard_score))
print("Soft voting score = {}".format(soft_score))

Ran score = 0.9766013763896242
Cat score = 0.977895855472901
XG score = 0.9770895205770047
Hard voting score = 0.9779333757691492
Soft voting score = 0.9776222292926079


In [322]:
"""
from sklearn.datasets import make_classification
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
# define dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# define the base models
level0 = list()
level0.append(('lr', LogisticRegression()))
level0.append(('knn', KNeighborsClassifier()))
level0.append(('cart', DecisionTreeClassifier()))
level0.append(('svm', SVC()))
level0.append(('bayes', GaussianNB()))
# define meta learner model
level1 = LogisticRegression()
# define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
# fit the model on all available data
model.fit(X, y)
"""

"\nfrom sklearn.datasets import make_classification\nfrom sklearn.ensemble import StackingClassifier\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.neighbors import KNeighborsClassifier\nfrom sklearn.tree import DecisionTreeClassifier\nfrom sklearn.svm import SVC\nfrom sklearn.naive_bayes import GaussianNB\n# define dataset\nX, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)\n# define the base models\nlevel0 = list()\nlevel0.append(('lr', LogisticRegression()))\nlevel0.append(('knn', KNeighborsClassifier()))\nlevel0.append(('cart', DecisionTreeClassifier()))\nlevel0.append(('svm', SVC()))\nlevel0.append(('bayes', GaussianNB()))\n# define meta learner model\nlevel1 = LogisticRegression()\n# define the stacking ensemble\nmodel = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)\n# fit the model on all available data\nmodel.fit(X, y)\n"

In [323]:
import math
import numpy as np
from sklearn.preprocessing import StandardScaler

dft = pd.read_csv('test.csv')
dft = dft.dropna()
dft = dft.drop(['pickup_time','drop_time'], axis = 1)

def deg_to_rad(dr):
    return (dr*math.pi)/180

dft['pick_lat'] = deg_to_rad(dft['pick_lat'])
dft['pick_lon'] = deg_to_rad(dft['pick_lon'])
dft['drop_lat'] = deg_to_rad(dft['drop_lat'])
dft['drop_lon'] = deg_to_rad(dft['drop_lon'])
dlon = dft['drop_lon'] - dft['pick_lon']
dlat = dft['drop_lat'] - dft['pick_lat']
dft['dist'] = np.sin(dlat/2)**2 + np.cos(dft['pick_lat'])*np.cos(dft['drop_lat'])*np.sin(dlon/2)**2
dft['dist'] = 2*np.arctan2(dft['dist']**0.5,(1-dft['dist'])**0.5)
R = 6373.0
dft['dist'] = R*dft['dist']
dft = dft.drop(['tripid','pick_lat','pick_lon','drop_lat','drop_lon'], axis = 1) 
X1 = dft

In [324]:
yhat = vot_soft.predict(X1)

In [325]:
df_test = pd.read_csv('test.csv')

In [326]:
df_test =df_test.drop(['meter_waiting_till_pickup','pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon'], axis = 1) 

In [327]:
df_new = df_test

In [328]:
df_new = df_new.drop(['additional_fare','duration','meter_waiting','meter_waiting_fare','fare'], axis = 1) 

In [329]:
df_new['prediction'] = yhat

In [330]:
df_new

,tripid,prediction
0,213284604,1
1,213286352,0
2,213293973,1
3,213294622,1
4,213298687,1
...,...,...
8571,222856243,1
8572,222857785,1
8573,222858416,1
8574,222858691,1


In [334]:
df_new['prediction'].value_counts()

1    8210
0     366
Name: prediction, dtype: int64

In [335]:
df_new.to_csv("hard.csv",index=False)